In [ ]:
import random
import numpy as np
import torch


In [ ]:
from main_ae import AgeModel
from main_ae import START_AGE, END_AGE, NUM_AGE_GROUPS


In [ ]:
NUM_AGE_GROUPS = 9
VALIDATION_RATE = 0.1
num_ages = END_AGE - START_AGE + 1


In [ ]:
random.seed(2019)
np.random.seed(2019)
torch.manual_seed(2019)


# AIHub dataset


In [ ]:
import json
import pandas as pd

In [ ]:
from fpt.data import join_face_df
from fpt.path import DTFR

DATA_CATEGORY = "aihub_family"
face_df = join_face_df(DTFR, DATA_CATEGORY)

In [ ]:
"c81fcbdb-157b-4283-841e-f1e84ae2f60a" in face_df.index  # False
with open("/home/jupyter/data/split/train_split.txt", "r") as f:
    lines = [line.rstrip() for line in f.readlines() if line.strip() in face_df.index]

In [ ]:
member_dict_path = "/home/jongphago/insightface/data/dict/member_dict.txt"
with open(member_dict_path, "r") as file:
    data = json.load(file)

In [ ]:
age_df = pd.read_csv("/home/jupyter/data/dataframe/df_aihub_ages.csv", index_col=0)
face_df = face_df.join(age_df, on="target")

# Model


In [ ]:
model = AgeModel(num_ages, NUM_AGE_GROUPS)  # age_pred, age_group_pred

# Dataloader


In [ ]:
from data import NiaDataset
import numpy as np
from PIL import Image
import os


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision


In [ ]:
train_meta_path = "nia_cropped/train_0.npy"
test_meta_path = "nia_cropped/test_0.npy"


In [ ]:
batch_size = 16


## NiaDataset


### Train dataloader


In [ ]:
transforms_train = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomApply(
            [
                torchvision.transforms.RandomAffine(degrees=10, shear=16),
                torchvision.transforms.RandomHorizontalFlip(p=1.0),
            ],
            p=0.5,
        ),
        torchvision.transforms.Resize((256, 256)),
        torchvision.transforms.RandomCrop((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)


In [ ]:
train_gen = NiaDataset(train_meta_path, transforms_train)


In [ ]:
train_loader = DataLoader(
    dataset=train_gen,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=0,
)


In [ ]:
train_iter = iter(train_loader)


In [ ]:
sample = next(train_iter)
sample.keys()


In [ ]:
for key in sample.keys():
    print(f"{key}:\t{sample[key][0]}")

In [ ]:
face_df.key

In [ ]:
idx = 1
file_name, _ = os.path.splitext(sample['file'][idx])
file_name in face_df.key

### Validation dataloader


In [ ]:
transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)
val_gen = NiaDataset(test_meta_path, transforms)
val_loader = DataLoader(
    val_gen, batch_size=1, shuffle=False, pin_memory=True, num_workers=0
)


# Optimizer


In [ ]:
from torch import optim
import torch
from torch.optim import lr_scheduler


In [ ]:
LAMBDA_1 = 0.2
LAMBDA_2 = 0.05
START_AGE = 0
END_AGE = 90
learning_rate = 1e-3
epoch = 2


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5, 8, 9], gamma=0.1)


# Loss function


In [ ]:
from mean_variance_loss import MeanVarianceLoss


In [ ]:
criterion1 = MeanVarianceLoss(LAMBDA_1, LAMBDA_2, START_AGE, END_AGE).cuda
criterion2 = torch.nn.CrossEntropyLoss().cuda()


# Train


In [ ]:
from main_ae import train_softmax, evaluate_softmax


In [ ]:
def train_softmax(train_loader, model, criterion2, optimizer, epoch, result_directory):
    model.cuda().train()
    running_loss = 0.0
    running_softmax_loss = 0.0
    interval = 1
    for i, sample in enumerate(train_loader):
        images = sample["image"].cuda()
        labels = sample["age_class"].cuda()
        _, output = model(images)
        loss = criterion2(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.data
        if (i + 1) % interval == 0:
            print("[%d, %5d] loss: %.3f" % (epoch, i, running_loss / interval))
            with open(os.path.join(result_directory, "log"), "a") as f:
                f.write("[%d, %5d] loss: %.3f\n" % (epoch, i, running_loss / interval))
            running_loss = 0.0


In [ ]:
for epoch in range(2):
    break
    train_softmax(train_loader, model, criterion2, optimizer, epoch, "result")
    loss_val, mae = evaluate_softmax(val_loader, model, criterion2)
    scheduler.step()


In [ ]:
# loss_val, mae


# Custom


In [ ]:
from torch import nn
from nia_age.main_ae import Embedding, AgeClassifier


In [ ]:
model = nn.Sequential(Embedding(), AgeClassifier(num_ages, NUM_AGE_GROUPS))
model = model.cuda()


In [ ]:
images = sample["image"]
images = images.cuda()


In [ ]:
pred_age, pred_age_group = model(images)
pred_age.shape, pred_age_group.shape


## Face Recognition


In [ ]:
from arcface_torch.losses import CombinedMarginLoss
from arcface_torch.configs.aihub_r50_onegpu import config as aihub_config
from arcface_torch.configs.base import config as cfg

cfg.update(aihub_config)
cfg.output = "work_dirs/aihub_r50_onegpu"

In [ ]:
margin_loss = CombinedMarginLoss(
    64,
    cfg.margin_list[0],
    cfg.margin_list[1],
    cfg.margin_list[2],
    cfg.interclass_filtering_threshold,
)

## Multi task Dataset


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

In [ ]:
class FaceAgeDataset(Dataset):
    def __init__(self, root_dir, face_df, transform=None):
        self.face_dataset = ImageFolder(root=root_dir, transform=transform)
        self.face_df = face_df

    def __len__(self):
        return len(self.face_dataset)

    def __getitem__(self, index):
        path, _ = self.face_dataset.samples[index]
        *_, key = os.path.splitext(path)[0].split("/")
        row = face_df.loc[key]
        age, family_id = row.age, row.family_id
        image, face_label = self.face_dataset[index]

        return image, face_label, age, family_id

In [ ]:
root_dir = "/home/jupyter/data/face-image/train_aihub_family"
face_age_dataset = FaceAgeDataset(root_dir, face_df)
iterator = iter(face_age_dataset)

In [ ]:
image, face_label, age, family_id = next(iterator)
image, face_label, age, family_id

## Face Recognition loss


In [ ]:
from torch.nn.functional import linear, normalize


In [ ]:
labels = labels.view(-1, 1)
norm_embeddings = normalize(embeddings)
norm_weight_activated = normalize(weight)
logits = linear(norm_embeddings, norm_weight_activated)
logits = logits.clamp(-1, 1)
